In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# 설치된 폰트를 Matplotlib에 적용
import matplotlib.font_manager as fm


# 폰트 경로 설정
font_dir = "/usr/share/fonts/truetype/noto"
font_files = fm.findSystemFonts(fontpaths=font_dir)
for font_file in font_files:
    fm.fontManager.addfont(font_file)

# 폰트 설정
plt.rcParams['font.family'] = 'Noto Sans CJK JP'


# 데이터 불러오기
file_path = 'data/train.csv'
try:
    data = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='euc-kr')

# 대화와 요약의 길이 계산
data['dialogue_length'] = data['dialogue'].apply(len)
data['summary_length'] = data['summary'].apply(len)

# 길이 분포 시각화
plt.figure(figsize=(12, 6))
sns.histplot(data['dialogue_length'], bins=50, color='blue', label='Dialogue Length')
sns.histplot(data['summary_length'], bins=50, color='red', label='Summary Length')
plt.legend()
plt.title('Dialogue and Summary Length Distribution')
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.show()

# 주제별 대화 수 시각화
plt.figure(figsize=(12, 6))
sns.countplot(y='topic', data=data, order=data['topic'].value_counts().index)
plt.title('Number of Dialogues per Topic')
plt.xlabel('Count')
plt.ylabel('Topic')
plt.show()

# WordCloud 생성 시 폰트 경로 지정
font_path = "/usr/share/fonts/truetype/noto/NotoSansCJKjp-Regular.otf"

# 워드 클라우드 생성
dialogue_text = " ".join(data['dialogue'].values)
summary_text = " ".join(data['summary'].values)

dialogue_wordcloud = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(dialogue_text)
summary_wordcloud = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(summary_text)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dialogue_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Dialogues')

plt.subplot(1, 2, 2)
plt.imshow(summary_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Summaries')

plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# 워드 클라우드 생성
topic_text = " ".join(data['topic'].values)

topic_wordcloud = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(topic_text)

plt.figure(figsize=(16, 8))
plt.subplot(1, 1, 1)
plt.imshow(topic_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Dialogues')


plt.show()

: 

In [ ]:
# 주제별 대화 수 시각화
plt.figure(figsize=(12, 6))
sns.countplot(y='topic', data=data, order=data['topic'].value_counts().index)
plt.title('Number of Dialogues per Topic')
plt.xlabel('Count')
plt.ylabel('Topic')
plt.show()

: 

In [ ]:
value_counts = data['topic'].value_counts()
print(value_counts)
print(type(value_counts))

# 막대 그래프 생성
value_counts[:40].plot(kind='bar')

# 그래프에 제목과 레이블 추가
plt.title('Value Counts of Topic')
plt.xlabel('Topics')
plt.ylabel('Counts')

# 그래프 보여주기
plt.show()

: 

In [ ]:
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

: 

In [ ]:
config_data = {
    "general": {
        "data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "wandb_repo",
        "project": "project_name",
        "name": "run_name"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

: 

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

: 

In [ ]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

: 

In [ ]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

: 

In [ ]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
loaded_config['general']['data_path'] = "/data/ephemeral/home/자연어처리대회/data_original/"

: 

In [ ]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

: 

In [ ]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

: 

In [ ]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

: 

In [ ]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "ssuksa"
loaded_config['wandb']['name'] = "NL_test"
loaded_config['wandb']['project'] = "NL_proj"

: 

In [ ]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

: 

In [ ]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

: 

In [ ]:
data.loc[599]['dialogue']

: 

In [ ]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

: 

: 

In [ ]:
'''
import urllib.request
import zipfile
import os

# Noto Sans CJK 폰트 다운로드
url = "https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip"
zip_path = "/tmp/NotoSansCJK.zip"
font_dir = "/usr/share/fonts/truetype/noto"

# 폰트 다운로드
urllib.request.urlretrieve(url, zip_path)

# 폰트 디렉토리 생성
os.makedirs(font_dir, exist_ok=True)

# 다운로드한 ZIP 파일을 추출
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(font_dir)

# 설치된 폰트를 Matplotlib에 적용
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 폰트 경로 설정
font_files = fm.findSystemFonts(fontpaths=font_dir)
for font_file in font_files:
    fm.fontManager.addfont(font_file)

# 폰트 설정
plt.rcParams['font.family'] = 'Noto Sans CJK JP'

# 테스트 그래프 그리기
plt.figure(figsize=(8, 4))
plt.text(0.5, 0.5, '한글 폰트 테스트', fontsize=20, ha='center')
plt.title('Noto Sans CJK JP 사용')
plt.show()
'''

: 